In [45]:
import glob
import os
import librosa
import numpy as np
import speechpy
from keras import layers, models
from keras.callbacks import TensorBoard
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split

DATA_DIR = "/Users/mbare/Documents/audana-dataset/dataset/Audio/"
SUB_DIRS = ["Truthful", "Deceptive"]


def find_max_mfccs_len(parent_dir,sub_dirs):
    lengths_mfccs = []
    for sub_dir in sub_dirs:
        for file_name in glob.glob(
            os.path.join(
                parent_dir,
                sub_dir,
                "*.wav")):
            X, sample_rate = librosa.load(file_name)
            mfccs = librosa.feature.mfcc(y=X, sr=sample_rate).T
            len_mfccs = mfccs.shape[0]
            lengths_mfccs.append(len_mfccs)
    max_len = sorted(lengths_mfccs,reverse=True)[0]
    print("Max MFCCS: " + str(max_len) + ". Will pad based on this")
    return max_len


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=10)
    mfccs = speechpy.processing.cmvnw(mfccs, win_size=301, variance_normalization=True)
    mfccs = np.mean(mfccs.T, axis=0)
    return mfccs


def parse_audio_files(parent_dir,sub_dirs):
    features, labels = np.empty((0, 10)), np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, 
                                         sub_dir, 
                                         "*.wav")):
            try:
                mfccs = extract_feature(fn)
            except Exception as e:
              print ("Error encountered while parsing file: ", fn)
              continue
            ext_features = np.hstack([mfccs])
            features = np.vstack([features,ext_features])
            labels = np.append(labels, [1 if sub_dir=="Deceptive" else 0])
    return np.array(features), np.array(labels, dtype = np.int)


In [46]:
features, labels = parse_audio_files(DATA_DIR, SUB_DIRS)

In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42)
model = Sequential()
model.add(Dense(16, kernel_initializer='uniform', activation='relu'))
model.add(Dense(2, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [48]:
tensorboard = TensorBoard(log_dir='./logs/run4_10mfccs/', 
                          write_graph=True, 
                          write_images=False)

model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_test, y_test),
          shuffle=True,
          verbose=False,
          callbacks=[tensorboard])

In [49]:
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25/25 [==============================] - 0s 28us/step
acc: 76.00%
